# House Price Prediction

In [1]:
# Necessary Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("Data/train.csv")
test_df = pd.read_csv("Data/test.csv")

In [40]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
df.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [6]:
for label,content in df.items():
    if pd.api.types.is_object_dtype(content):
        df[label] = content.astype("category")

In [7]:
categorical_columns = df.select_dtypes(include=['category']).columns
numerical_columns = df.select_dtypes(exclude=['category']).columns

In [26]:
X = df.drop("SalePrice",axis=1)
Y = df["SalePrice"]

In [9]:
numerical_columns = numerical_columns.drop("SalePrice")

In [10]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor

In [28]:
numeric_pipeline = Pipeline(steps = [("imputer",SimpleImputer(strategy="median")),
                                    ("scaler",StandardScaler())])
categorical_pipeline = Pipeline(steps = [("imputer",SimpleImputer(strategy="most_frequent")),
                                        ("one_hot",OneHotEncoder(handle_unknown="ignore"))])

preprocessor = ColumnTransformer(transformers=[("num",numeric_pipeline,numerical_columns),
                                               ("cat",categorical_pipeline,categorical_columns)])
X_train = preprocessor.fit_transform(X)
Y_train = Y
X_test = preprocessor.transform(test_df)

In [29]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

model = RandomForestRegressor()
model.fit(X_train,Y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [30]:
from sklearn.model_selection import RandomizedSearchCV

In [31]:
model = GradientBoostingRegressor()
grid = {"n_estimators":np.arange(500,2000,10),
       "min_samples_split":np.arange(2,10,1),
       "min_samples_leaf":np.arange(2,10,1),
       "max_features":["sqrt","log2"]}
rs_gradient_model = RandomizedSearchCV(estimator=model,
                                       cv=5,
                                       n_iter=10,
                                       param_distributions=grid,
                                       verbose=True)
rs_gradient_model.fit(X_train,Y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


,estimator,GradientBoostingRegressor()
,param_distributions,"{'max_features': ['sqrt', 'log2'], 'min_samples_leaf': array([2, 3, ..., 6, 7, 8, 9]), 'min_samples_split': array([2, 3, ..., 6, 7, 8, 9]), 'n_estimators': array([ 500, ..., 1980, 1990])}"
,n_iter,10
,scoring,None
,n_jobs,None
,refit,True
,cv,5
,verbose,True
,pre_dispatch,'2*n_jobs'
,random_state,None
,error_score,nan


In [32]:
rs_gradient_model.best_params_

{'n_estimators': np.int64(1600),
 'min_samples_split': np.int64(5),
 'min_samples_leaf': np.int64(5),
 'max_features': 'sqrt'}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid2 = {"n_estimators":np.arange(1000,1500,100),
        "min_samples_split":[6,8,10],
        "min_samples_leaf":[2,4,6],
        "max_features":["log2","sqrt"]}
gs_gradient_model = GridSearchCV(param_grid=grid2,cv=5,estimator=model,verbose=True)
gs_gradient_model.fit(X_train,Y_train)

In [ ]:
gs_gradient_model.score(X_test,Y_test)

In [37]:
ideal_model = GradientBoostingRegressor(n_estimators=1600,
                                       min_samples_split=5,
                                       min_samples_leaf=5,
                                       max_features="sqrt",random_state=32)
ideal_model.fit(X_train,Y_train)

,loss,'squared_error'
,learning_rate,0.1
,n_estimators,1600
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,5
,min_samples_leaf,5
,min_weight_fraction_leaf,0.0
,max_depth,3
,min_impurity_decrease,0.0
,init,None


In [38]:
y_preds = ideal_model.predict(X_test)

In [39]:
y_preds

array([126817.26148595, 161415.30431303, 188326.69238183, ...,
       173466.14636907, 121815.36312615, 213901.23504208], shape=(1459,))

In [45]:
sample = pd.DataFrame(columns = ["Id","SalePrice"])
sample["Id"] = test_df.Id
sample["SalePrice"] = y_preds

In [48]:
sample.to_csv("Data/Sample_submission.csv",index=False)

In [49]:
pd.read_csv("Data/sample_submission.csv")

,Id,SalePrice
0,1461,126817.261486
1,1462,161415.304313
2,1463,188326.692382
3,1464,202159.460854
4,1465,171316.431617
...,...,...
1454,2915,79869.918350
1455,2916,75963.674517
1456,2917,173466.146369
1457,2918,121815.363126
